# Clustering on PBMC3k dataset
Links: 
- [The PBMC3k dataset from 10Xgenomics](http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz)
- [Scanpy tutorial](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html)
- [Tutorial Seurat (R)](https://satijalab.org/seurat/articles/pbmc3k_tutorial.html)

## Preprocessing using scanpy

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [ ]:
sc.settings.verbosity = 2             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
adata = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

adata.var_names_make_unique()

In [ ]:
# Filtering
sc.pp.filter_cells(adata, min_genes=200)    # keep only cells that have at least 200 genes
sc.pp.filter_genes(adata, min_cells=3)      # keep only genes that were found in at least 3 cells

adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)  # compute several metrics

adata = adata[adata.obs.n_genes_by_counts < 2500, :]  # remove cells with overall less than 2500 gene counts
adata = adata[adata.obs.pct_counts_mt < 5, :]  # remove cells with high percentage of mitochondrial genes (poor-quality cells)

sc.pp.normalize_total(adata, target_sum=1e4)  # normalize counts to 10,000 counts per cell

In [ ]:
# Extracting highly variable genes
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)  # higly variable genes
adata = adata[:, adata.var.highly_variable]

# regress out effects and scale each gene to unit variance
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)

## Clustering
Continue with the data as pandas DataFrame 

In [ ]:
genes = sc.get.var_df(adata)
cells = sc.get.obs_df(adata)  # not used

df = pd.DataFrame(adata.X)
df.columns = list(genes.reset_index()['index'])

df.head()

In [ ]:
# PCA with 20 components, create explained variance plot
pca = PCA(n_components=20)
pca.fit(df.T)

plt.plot(pca.explained_variance_ratio_)
plt.ylabel('Explained Variance')
plt.xlabel('Principal Component')
plt.title('Explained variance of PCA')
plt.show()

In [ ]:
# Continue with 7 components
X = pca.components_[:7].T
print(X.shape)

In [ ]:
# Embed X in 2D space using t-SNE
X_embedded = TSNE().fit_transform(X)

In [ ]:
sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], legend = None, size = 4, color = 'lightgrey')
plt.xlabel('t-SNE 1')
plt.xticks([])
plt.ylabel('t-SNE 2')
plt.yticks([])
plt.title('De PBMC-cellen in 2D d.m.v. t-SNE')
plt.show()

In [ ]:
sscores = []
for n_clust in range(2, 12):
    cl = KMeans(n_clusters = n_clust, n_init = 10, random_state = 42).fit(X)
    cl_labels = cl.labels_
    sscores.append(silhouette_score(X, cl_labels))

sns.lineplot(x = [i for i in range(2, 12)], y = sscores)
plt.xlabel('Aantal clusters')
plt.ylabel('Silhouette score')
plt.title('Silhouette scores voor verschillende\naantallen clusters')
plt.show()

In [ ]:
cl = KMeans(n_clusters=8, n_init = 10, random_state = 42).fit(X)
cl_labels = cl.labels_

sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], hue = cl_labels, legend = None, size = 4, palette = 'mako')
plt.xlabel('t-SNE 1')
plt.xticks([])
plt.ylabel('t-SNE 2')
plt.yticks([])
plt.title('K-Means clustering op de PBMC-cellen')
plt.show()

In [ ]:
gene = 'CST3'
sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], hue = df[gene], legend = None, size = 4, palette = 'mako_r')
plt.xlabel('t-SNE 1')
plt.xticks([])
plt.ylabel('t-SNE 2')
plt.yticks([])
plt.title(f'Expressie van het {gene}-gen in de PBMC-cellen')
plt.show()

In [ ]:
gene = 'MS4A1' 
sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], hue = df[gene], legend = None, size = 4, palette = 'mako_r')
plt.xlabel('t-SNE 1')
plt.xticks([])
plt.ylabel('t-SNE 2')
plt.yticks([])
plt.title(f'Expressie van het {gene}-gen in de PBMC-cellen')
plt.show()

In [ ]:
gene = 'GNLY' 
sns.scatterplot(x = X_embedded[:,0], y = X_embedded[:,1], hue = df[gene], legend = None, size = 4, palette = 'mako_r')
plt.xlabel('t-SNE 1')
plt.xticks([])
plt.ylabel('t-SNE 2')
plt.yticks([])
plt.title(f'Expressie van het {gene}-gen in de PBMC-cellen')
plt.show()